# Functores

Pasos para entender una álgebra en Haskell (según Chris Allen):
1. Encuentra un patrón y hazlo general.
2. Define cuáles son las leyes que va a cumplir.
3. Dale un nombre cool.
4. Pregúntate cómo pudiste vivir todo este tiempo sin él.

## Buscando un patrón
Define una función que aplica una función a todos los elementos de una lista.

In [1]:
map :: (a -> b) -> [a] -> [b]
map _ []     = []
map f (x:xs) = f x : map f xs

In [11]:
map (+1) [0..4]
map id   [0..4]
map ((+1) . (*2)) [0..4]
(map (+1) . map (*2)) [0..4]

Line 2: Redundant map
Found:
map id
Why not:
idLine 4: Use map once
Found:
map (+ 1) . map (* 2)
Why not:
map ((+ 1) . (* 2))

[1,2,3,4,5]

[0,1,2,3,4]

[1,3,5,7,9]

[1,3,5,7,9]

Define una función que aplica una función al elemento envuelto en un `Maybe a`. Recuerda la definición de `Maybe`:

```haskell
data Maybe a = Nothing | Just a
```

In [3]:
import Data.Maybe

mapMaybe :: (a -> b) -> Maybe a -> Maybe b
mapMaybe _ Nothing  = Nothing
mapMaybe f (Just x) = Just (f x)

In [8]:
mapMaybe (*2) (Just 4)
mapMaybe (*2) Nothing
mapMaybe id   (Just 5)

Line 3: Use catMaybes
Found:
mapMaybe id
Why not:
catMaybes

Just 8

Nothing

Just 5

Define una función que aplica otra función al segundo elemento de una dupla.

In [5]:
mapDupla :: (b -> c) -> (a, b) -> (a, c)
mapDupla f (a, b) = (a, f b)

In [7]:
mapDupla (++[1,2]) (['a', 'b'], [4,5])
mapDupla id (['a', 'b'], [4,5])

("ab",[4,5,1,2])

("ab",[4,5])

## Definición

Diremos que un Functor es aquel tipo de dato que se puede mapear. Es decir, podemos aplicar una función al contenido del tipo de dato, sin alterar la estructura que lo contiene.

![Ilustración de Functores](functor_1.png)

A la función que mapea sobre un functor, la llamaremos `fmap`. Además, esperamos que se cumplan algunas reglas:

1. Identidad: `fmap id == id`.
2. Composición: `fmap (f . g) == fmap f . fmap g`.

Utilizando algo llamado el _"Teorema libre para fmap"_ y la propiedad 1, podemos demostrar que siempre se cumple la propiedad 2. Por lo tanto, basta revisar que se cumple lo primero.

$$ \operatorname{fmap}\;(id\;.\;g) = \operatorname{fmap}\;g = id\;.\; \operatorname{fmap}\;g = \operatorname{fmap}\;id\;.\;\operatorname{fmap}\;g $$
$$ \operatorname{fmap}\;(f\;.\;id) = \operatorname{fmap}\;f = \operatorname{fmap}\;f\;.\;id = \operatorname{fmap}\;f\;.\;\operatorname{fmap}\;id $$

Define la clase `Functor` y la función `fmap`.

In [12]:
class Functor f where
    fmap :: (a -> b) -> f a -> f b

Haz que las listas sean instancias de la clase para functores.

In [13]:
instance Functor [] where
    fmap = map

In [21]:
fmap (+1) [0..4]
fmap id   [0..4]
fmap ((+1) . (*2)) [0..4]
(fmap (+1) . fmap (*2)) [0..4]

Line 2: Functor law
Found:
fmap id
Why not:
idLine 4: Functor law
Found:
fmap (+ 1) . fmap (* 2)
Why not:
fmap ((+ 1) . (* 2))

[1,2,3,4,5]

[0,1,2,3,4]

[1,3,5,7,9]

[1,3,5,7,9]

Haz que el tipo `Maybe` sea instancia de la clase de functores.

In [14]:
instance Functor Maybe where
    fmap = mapMaybe

In [22]:
fmap (*2) (Just 4)
fmap (*2) Nothing
fmap id   (Just 5)

Line 3: Functor law
Found:
fmap id
Why not:
id

Just 8

Nothing

Just 5

Haz que las duplas sean instancias de la clase de functores en su segunda entrada.

In [17]:
instance Functor ((,) a) where
    fmap = mapDupla

In [23]:
fmap (++[1,2]) (['a', 'b'], [4,5])
fmap id (['a', 'b'], [4,5])

Line 2: Functor law
Found:
fmap id
Why not:
id

("ab",[4,5,1,2])

("ab",[4,5])

Haz que las funciones sean instancias de functores con respecto a la segunda función.

In [20]:
-- (a -> b) -> (r -> a) -> (r -> b)
instance Functor ((->) r) where
    fmap = (.)

In [25]:
fmap (+2) (*5) 1

7

### Operadores

La función `fmap` recibe exactamente 2 argumentos, por lo tanto, puede usarse como operador:

```haskell
(+5) `fmap` [1,2,3,4]
```

Sin embargo, se ve muy feo lo anterior, y puede no ser legible.

Crea un operador infijo, con asociatividad izquierda y jerarquía 4 que sea equivalente a `fmap`. Dale el nombre `<$>`.

In [26]:
-- (a -> b) -> f a -> f b : <$>
-- (a -> b) ->   a ->   b :  $

infixl 4 <$>
(<$>) = fmap

In [29]:
:t (<$)

(<$) :: forall (f :: * -> *) a b. Functor f => a -> f b -> f a

## Utilizando functores

- Crear funciones del tipo `f a -> f b`.
- Aplicar series de modificaciones.

In [ ]:
-- f a -> f b : g
-- f b -> f c : h

-- g <$> h : f a -> f c

Crea una función que recibe algo de tipo `Maybe Int` y te devuelve algo de tipo `Maybe Bool`, donde el booleano es `True` si el número es 0 o positivo, y `False` en otro caso. Primero, escríbela usando caza de patrones, y luego reescríbela con Functores.

In [30]:
-- Maybe a = Nothing | Just a

maybePositivo :: Maybe Int -> Maybe Bool
maybePositivo Nothing  = Nothing
maybePositivo (Just x) = Just (x >= 0)

In [31]:
maybePositivo' :: Maybe Int -> Maybe Bool
maybePositivo' mi = (\x -> x >= 0) <$> mi

In [32]:
maybePositivo  Nothing
maybePositivo' Nothing

maybePositivo  (Just 10)
maybePositivo' (Just 10)

maybePositivo  (Just (-4))
maybePositivo' (Just (-4))

Nothing

Nothing

Just True

Just True

Just False

Just False

Crea una función que toma una lista de `Maybe`s con un elemento de tipo numérico, y te regresa el doble de todos los números.

In [45]:
--[Nothing, Just 5, Just 10] -> [Nothing, Just 10, Just 20]

dobleMaybes :: Num a => [Maybe a] -> [Maybe a]
--dobleMaybes = fmap (fmap (*2))
dobleMaybes = (fmap . fmap) (*2)

In [43]:
dobleMaybes [Nothing, Just 4, Just 6]

[Nothing,Just 8,Just 12]

In [46]:
:t (fmap . fmap)

(fmap . fmap) :: forall (f1 :: * -> *) (f2 :: * -> *) a b. (Functor f1, Functor f2) => (a -> b) -> f1 (f2 a) -> f1 (f2 b)

## Limitantes

- Solo podemos ir de un functor a otro functor (posiblemente con functores anidados).
    - `liftF2 :: Functor f => (a -> b -> c) -> f a -> f b -> f c`
- Cada tipo de dato solo tiene un functor (es decir, los functores son únicos).

In [ ]:
sumaJust :: Maybe a -> Maybe b -> Maybe c
sumaJust (Just x) (Just y) = Just (x + y)

## Usos

- Modificaciones dentro de contextos: Bases de datos.
- Cambios de contexto (Transformaciones Naturales).
- Secuencias de modificaciones que pueden fallar.

Tipos de datos que son instancias de functores:
- Listas.
- Árboles (binarios, rojinegros, tries, etc).
- Matrices.
- Grafos.
- Funciones.

In [48]:
data Estudiante = Estudiante { nombre   :: String
                             , promedio :: Double
                             , inscrito :: Bool
                             } deriving Show

type BDDEstudiantes = [Estudiante]

inscritos :: BDDEstudiantes -> [Maybe Estudiante]
inscritos = fmap (\estudiante -> if inscrito estudiante then Just estudiante else Nothing)

puntoExtra :: [Maybe Estudiante] -> [Maybe Estudiante]
puntoExtra = (fmap . fmap) (\estudiante -> estudiante { promedio = promedio estudiante + 1 })

actualizaCalificaciones :: BDDEstudiantes -> [Maybe Estudiante]
actualizaCalificaciones = puntoExtra <$> inscritos

In [49]:
actualizaCalificaciones [ Estudiante "Alonzo Church" 9   True
                        , Estudiante "Alan Turing"   8.3 True
                        , Estudiante "Kurt Godel"    9   False
                        ]

[Just (Estudiante {nombre = "Alonzo Church", promedio = 10.0, inscrito = True}),Just (Estudiante {nombre = "Alan Turing", promedio = 9.3, inscrito = True}),Nothing]